In [1]:
import numpy as np
import torch
from Utilities import Utilities as Utils

In [3]:
# Model hyperparameters
Filters = 128
Layers = 13
HistoryDepth = 8
TargetDevice = "cuda"
model_name = "test2"
target_path = '../../Models/scripted/'

resnetModel, policyModel, valueModel = Utils.modelLoader(model_name, Filters, Layers, HistoryDepth, TargetDevice)

resnetExample = np.zeros((1, HistoryDepth + 1, 15, 15), dtype=np.float32) 
resnetExample = torch.tensor(resnetExample)
resnetExample = resnetExample.to(TargetDevice)

headsExample = np.zeros((1, Filters, 15, 15), dtype=np.float32)
headsExample = torch.tensor(headsExample)
headsExample = headsExample.to(TargetDevice)

traced_resnet = torch.jit.trace(resnetModel, resnetExample)
traced_polhead = torch.jit.trace(policyModel, headsExample)
traced_valhead = torch.jit.trace(valueModel, headsExample)

traced_resnet.save(f'{target_path}/ResNet/{model_name}.pt')
traced_polhead.save(f'{target_path}/PolHead/{model_name}.pt')
traced_valhead.save(f'{target_path}/ValHead/{model_name}.pt')